In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **READING THE DATA**

In [ ]:
import numpy as np
import pandas as pd

df = pd.read_csv('../input/loan-predication/train_u6lujuX_CVtuZ9i (1).csv')
df.head()

In [ ]:
df.shape

# **CHECKING WHETHER THE DATA HAS ANY NULL VALUES OR NOT**

In [ ]:
df.isnull().sum()

**WE CAN SEE THAT SOME COLUMNS HAVE THE NULL VALUES**

In [ ]:
df.dtypes ## by using .dtypes we will get the data type of each column

# **DEALING WITH THE MISSING VALUES**

***'GENDER'' COLUMN HAS 13 NULL VALUES.***

In [ ]:
import seaborn as sns
sns.countplot('Loan_Status',hue='Gender',data=df)

**BY ANALYSING THE GRAPH WE SEE THAT THE MAJORITY OF GENDER IS MALE SO, WE ARE GOING TO FILL GENDER COLUMN'S NULL VALUES WITH 'MALE'**

In [ ]:
df['Gender'] = df['Gender'].fillna('Male')

***'MARRIED'COLUMN HAS 3 NULL VALUES***

In [ ]:
sns.countplot('Loan_Status',hue='Married',data=df)

**WE CAN UNDERSTAND THAT MARRIED PEOPLE WILL GET THE LOAN. SO AS THE MAJORITY SAYS WE WILL FILL THE 'MARRIED' COLUMN'S NULL VALUES WITH 'YES'**

In [ ]:
df['Married'] = df['Married'].fillna('Yes')

***'DEPENDENTS' COLUMN HAS 15 NULL VALUES***

In [ ]:
sns.countplot('Loan_Status',hue='Dependents',data=df)

**Since most of the dependents are 0 we are gonna fill with it with 0**

In [ ]:
df['Dependents'] = df['Dependents'].fillna('0')

***'SELF-EMPLOYED' COLUMN HAS 32 NULL VALUES***

In [ ]:
sns.countplot('Loan_Status' , hue = 'Self_Employed' , data = df)

**WE CAN SEE HERE THAT THE 'SELF_EMPLOYED' COLUMN' HAS THE MAJORITY 'NO'. SO WE ARE GOING TO FILL THAT COLUMN WITH 'NO'**

In [ ]:
df['Self_Employed']=df['Self_Employed'].fillna('No')

***NOW THE DEALING WITH THE 'LOANAMOUNT' COLUMN WHICH HAS 22 NULL VALUES.***

In [ ]:
sns.distplot(df['LoanAmount'])

In [ ]:
sns.scatterplot(df['LoanAmount'],y=np.arange(0,614))

**THE OUTLIERS IN THE DATA SEEMS TO BE RIGHT SKEWED ,  SO WE WILL NOT FILL THE VALUES BY THE MEAN OF THIS DATA, WE ARE GOING TO TAKE THE MEAN WHICH DOES NOT CONTAIN OUTLIERS.**

In [ ]:
mean=df[df['LoanAmount']<=400]['LoanAmount'].mean()
df['LoanAmount'].fillna(mean,inplace=True)

***'LOAN_AMOUNT_TERM' COLUMN HAS 14 NULL VALUES***

In [ ]:
sns.scatterplot(df['Loan_Amount_Term'],y=np.arange(0,614))

**TO FILL THE MISSING VALUES WE WILL USE MODE**

In [ ]:
df['Loan_Amount_Term'].fillna(df['Loan_Amount_Term'].value_counts().idxmax(), inplace=True)

***'CREDIT_HISTORY' HAS 50 NULL VALUES***

In [ ]:
df['Credit_History'].unique()

In [ ]:
sns.countplot('Loan_Status',hue='Credit_History',data=df)

**TO FILL THE MISSING VALUES WE WILL USE MODE**

In [ ]:
df['Credit_History'].fillna(df['Credit_History'].value_counts().idxmax(), inplace=True)

**NOW WE WILL CHECK WHETHER THE DATA STILL HAS ANY NULL VALUES OR NOT**

In [ ]:
df.isnull().sum()

In [ ]:
df.head(10)

# **ONE-HOT ENCODING WILL BE DONE TO CHANGE THE CATEGORICAL DATA INTO NUMERICAL DATA BEFORE WE DEPLOY IT INTO OUR MODEL SO THAT THE PROCESS BECOMES EASY.**

In [ ]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
df_data = df.copy()

**CHECKING WHETHER ALL OUR DUPLICATE VALUES HAVE BEEN REMOVED OR NOT, SUCH AS yes,YES,no,NO......**

In [ ]:
print(df_data['Gender'].unique())
print(df_data['Married'].unique())
print(df_data['Education'].unique())
print(df_data['Self_Employed'].unique())
print(df_data['Property_Area'].unique())

**TRANSFORMING THE CATEGORICAL DATA INTO NUMERICAL DATA 0,1,2..**

In [ ]:
df_data['Gender'] = lb.fit_transform(df_data['Gender'])
df_data['Married'] = lb.fit_transform(df_data['Married'])
df_data['Education'] = lb.fit_transform(df_data['Education'])
df_data['Self_Employed'] = lb.fit_transform(df_data['Self_Employed'])
df_data['Property_Area'] = lb.fit_transform(df_data['Property_Area'])
df_data['Loan_Status'] = lb.fit_transform(df_data['Loan_Status'])
df_data['Dependents'] = lb.fit_transform(df_data['Dependents'])

**LET US CHECK IT**

In [ ]:
print(df_data['Gender'].unique())
print(df_data['Married'].unique())
print(df_data['Education'].unique())
print(df_data['Self_Employed'].unique())
print(df_data['Property_Area'].unique())
print(df_data['Loan_Status'].unique())
print(df_data['Dependents'].unique())

In [ ]:
df_data.head()

**YOU CAN SEE THAT THE DATA HAS BEEN TRANSFROMED COMPLETELY INTO THE NUMERICAL DATA**

# **DEPLOYING THE DATA INTO OUR MODEL**

In [ ]:
df_data.columns

# **STEP_1:- INITIALISATION OF X and Y**

In [ ]:
X = df_data.drop(['Loan_ID','Loan_Status'],axis=1)
Y = df_data[['Loan_Status']]

# **STEP_2:- SPLITING TRAINING AND TESTING SETS**

In [ ]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(X,Y,test_size=0.3,random_state=43)

# **STEP_3 :- UNDERGOING STANDARDISATION PROCESS**

***BECAUSE OUR DATA IS SO BIG***

In [ ]:
from sklearn.preprocessing import StandardScaler
std = StandardScaler()
std.fit(xtrain)
xtr = std.transform(xtrain)

# **STEP_4:- TRAINING OUR MODEL ON TRAINING DATA**

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
lgt = LogisticRegression()


In [ ]:
lgt.fit(xtr,ytrain)


# **STEP_5:- PREDICTING ON TEST VALUES**

***TRANSFORMING THE TESTING FEATURES INTO STANDARD FORMS***

In [ ]:
xts = std.transform(xtest)

***NOW PREDICTING ON TRANSFORMED TESTING FEATURES.***

In [ ]:
predict = lgt.predict(xts)

In [ ]:
print(predict)

***GETTING THE ACCURACY REPORT***

In [ ]:

print(classification_report(ytest,predict))

# **HENCE,THE ACCURACY LEVEL IS OF ABOUT 86%**